In [1]:
#Import packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random

#Import data frames
df = pd.read_excel("model.xlsx", encoding = "ISO-8859-1")
#General variables
Conv_MJ_kwh = 3.6 # Conversion factor between MJ and kWh
Pop_2010 = 462124 #Based on the cantonal statistics
Pop_rate = 0.01 #Based on the cantonal statistics: Average of the population's evolution from 2000 to 2018
S_df = 19171151 #Sum of the SRE in m2 in the data frame
year = [] # Array  to store the year from 20011 to 2050
x = 2011 # Set the time to 2011
sim = 0 # Set the number of simulation to 0
    # Function to fill the list: "year", from 2011 to 2050
while x <= 2050:
    year.append(x)
    x += 1

x = 2011 # Reset the time to 2011
df_E_S = pd.DataFrame(year, columns= ["Year"]) # Create a data frame to
df_kwh_m2 = pd.DataFrame(year, columns= ["Year"]) # Create a dataframe in order to store the energy demand per square meter over the years and the simulation
df_RR = pd.DataFrame(year, columns= ["Year"]) # Create a dataframe to store the retrofit rate over the year and the simulation

#Energy variables
IDC_n = 40 #Energy consumed per m2 for new buildings. Based on ENDK, Consommation Energétique des Bâtiments, Berne 2014.
#According to the 2014_MOPEC and updates: A new MOPEC should be created in 2025 and then will have major updates every 5 to 10 years.
DIDC = 0 #Variation of the IDC through years in case the legislation vary.

CECB_A = 31.0 # everything under this value is in the category A, according to the CECB classification
CECB_B = 63.0 # everything under this value is in the category B, according to the CECB classification
CECB_C = 94.0 # everything under this value is in the category C, according to the CECB classification
CECB_D = 126.0 # everything under this value is in the category D, according to the CECB classification
CECB_E = 157.0 # everything under this value is in the category E, according to the CECB classification
CECB_F = 189.0 # everything under this value is in the category F, according to the CECB classification
CECB_G = 221.0 # everything under this value is in the category G, according to the CECB classification
Coef_A = 0
Coef_B = 1
Coef_C = 2
Coef_D = 3
Coef_E = 4
Coef_F = 5
Coef_G = 6
Category = ['A', 'B', 'C', 'D', 'E', 'F', 'G'] # CECB categories
IDC_cat = [31, 63, 94, 126, 157, 189, 221] # Energy consumed per square meter for every CECB categories.
df_per_cat = pd.DataFrame(Category, columns= ["Category"]) # Creation of a dataframe in order to store the percentage of surface per CECB categories, for every year.

#Surface-Human Ratio
SHR_ini = 45 # m2/capita
D_SHR = 0 #variation of the SHR through years
SHR = SHR_ini # We set the SHR to the initial SHR


#Retrofit rates
#Reference for the specific retrofit rate: "GAPxPLORE: Energy Performance Gap in existing, new, and renovated buildings: Learning from large-scale datasets".
RR_tot_ini = 0.008
dif_RR = 0.001
RR_max = 0.012
RR_A = 0.000
RR_B = 0.010
RR_C = 0.031
RR_D = 0.112
RR_E = 0.210
RR_F = 0.224
RR_G = 0.414

diff_prop = 0 #Variation in the proportion of building who is retrofitted in CECB class A and B.

#Clean and prepare the data frame
df['INDICE'] = df['INDICE']/Conv_MJ_kwh # Conversion of the energy index from MJ to kWh
df['CONSO_CORRIGEE'] = df['INDICE'] * df['SRE'] # Transformation of the coulmn energy consumption in kWh
Index = [] # Creation of the list Index
Index = df['INDICE'].values.tolist() 
Coef = [] # Creation of the list Coef in order to store the coef of every CECB depending of the index of each building.

    # For every building, we screen the index and add one coef corresponding to its CECB class.
for step in Index:
    if step > CECB_F:
        Coef.append(Coef_G)
    elif step < CECB_F and step > CECB_E:
        Coef.append(Coef_F)
    elif step < CECB_E and step > CECB_D:
        Coef.append(Coef_E)
    elif step < CECB_D and step > CECB_C:
        Coef.append(Coef_D)
    elif step < CECB_C and step > CECB_B:
        Coef.append(Coef_C)
    elif step < CECB_B and step > CECB_A:
        Coef.append(Coef_B)
    elif step < CECB_A:
        Coef.append(Coef_A)

df['COEFFICIENT'] = Coef

conso_corige = []
conso_corige = df['CONSO_CORRIGEE'].values.tolist()
E_initial = sum(conso_corige)/ S_df # E_initial is the energy consumed per square meter in 2010 from the building of the data base.

#print(E_initial)
#print(df)

130.84055645738306
          EGID AVANT_93  ANNEE     SRE      INDICE  CONSO_CORRIGEE  \
0      1016168      Oui   2010  1708.0  130.277778   222514.444444   
1      1010414      Oui   2010  1200.0   81.111111    97333.333333   
2      1001062      Oui   2010   960.0  130.000000   124800.000000   
3      1014852      Oui   2010  2103.0  134.722222   283320.833333   
4      1009254      Oui   2010   697.0  143.888889   100290.555556   
...        ...      ...    ...     ...         ...             ...   
11222  2033865      Oui   2010   534.0  126.944444    67788.333333   
11223  1014561      Oui   2010  4104.0  121.388889   498180.000000   
11224  1009757      Oui   2010   715.0  172.222222   123138.888889   
11225  1006214      Oui   2010   623.0  136.666667    85143.333333   
11226  1027416      Oui   2010   219.0  294.166667    64422.500000   

       EMISSION_CO2  COEFFICIENT  
0             58512            4  
1             25612            2  
2             25761            4  


In [2]:
for sim in range(0,1): # We can select the number of simulation we want.

    #Restart from 0 the dataframes
    if len(df_per_cat.columns) >= 1:
        df_per_cat = df_per_cat.drop(df_per_cat.columns,axis=1)
        
    #Variables declaration
        #Population
    Pop_x = Pop_2010
        #Surface
        #Creation of empty list in order to store the total surface in every CECB categories.
    SRE_A = []
    SRE_B = []
    SRE_C = []
    SRE_D = []
    SRE_E = []
    SRE_F = []
    SRE_G = []
    SRE = [] # A list in order to store the SRE from the df
    SRE = df['SRE'].values.tolist()
    Surface_t = [] # List to store temporarilly the surface.
    S_n_t = 0
    S_n = 0
    range = 0.01
    S_add = Pop_2010*SHR - S_df # The additional surface is the surface total in 2010 - the surface of the df.
    S_A = 0
    S_B = 0
    S_C = 0
    S_D = 0
    S_E = 0
    S_F = 0
    S_G = 0
    S_tot_year = []
    sum_A = 0
    sum_B = 0
    sum_C = 0
    sum_D = 0
    sum_E = 0
    sum_F = 0
    sum_G = 0
    per_CECB = []
    
        #Energy
        #Creation of empty list in order to store the IDC in every CECB categories.
    IDC_A = []
    IDC_B = []
    IDC_C = []
    IDC_D = []
    IDC_E = []
    IDC_F = []
    IDC_G = []
    IDC = [] # A list in order to store the IDC from the df
    IDC = df['INDICE'].values.tolist()
    IDC_t = [] # List to store temporarilly the IDC.
    IDC_n = 40 # We set the regulation for new building at 40 kWh / m2
    E_df_year = [] # We create a list to store the total energy consumed every year by the building of the df.
    E_add_year = [] # We create a list to store the total energy consumed every year by the building which are not in the df.
    E_n = 0 #Yearly energy consummed by the new surface.
    E_n_year = [] #List of the new energy consummed by the new surface every year.
    E_S = [] # A list in order to store the total average of energy consumption per square meter through years
    E_tot = []
        
        # Proportion of building of every CECB class which are retrofitted in category A. We assume that the rest of the retrofitted building foes to the class B
    prop_G = 0.06
    prop_F = 0.06
    prop_E = 0.06
    prop_D = 0.10
    prop_C = 0.30
    prop_B = 1.00
    
        #Retrofit rate
    RR_year = [] #A list in order to store every retrofit rate through years
    RR_tot = RR_tot_ini
    
        # Declaration of counters
    number = 0
    count = 0
    number_A = 0 # Number of building which will be retrofitted in A instead of B
    
    
    df_per_cat['Categories'] = Category #Add the coulumn "category" to the df who will store the evolution of the surface percentage per CECB category-
    
    #Sort the surface and the IDC by CECB categories
    i = 0
    for step in IDC:
        if step > 189:
            SRE_G.append(SRE[i])
            IDC_G.append(IDC[i])
        elif step < 189 and step > 157:
            SRE_F.append(SRE[i])
            IDC_F.append(IDC[i])
        elif step < 157 and step > 126:
            SRE_E.append(SRE[i])
            IDC_E.append(IDC[i])
        elif step < 126 and step > 94:
            SRE_D.append(SRE[i])
            IDC_D.append(IDC[i])
        elif step < 94 and step > 63:
            SRE_C.append(SRE[i])
            IDC_C.append(IDC[i])
        elif step < 63 and step > 31:
            SRE_B.append(SRE[i])
            IDC_B.append(IDC[i])
        elif step < 31:
            SRE_A.append(SRE[i])
            IDC_A.append(IDC[i])
        i += 1
    
    x = 2011
    while x <= 2050:
        
    
        E_df = 0 #Reset to 0 the energy consummed by the dataframe.
        
        
        #--- Phase 0: Preliminary preparation ---#
        
        #Measurement of the Pop_x of the year
        Pop_x = Pop_x + Pop_x * Pop_rate #Population of the year x is the population of the previous year + pop previous year times the growing rate.
        #Measurement of the surface tot of the year
        SHR = SHR + D_SHR
        S_tot = Pop_x * SHR_ini
        S_tot_year.append(S_tot) #We store the total surface of every year in a list
        
        #Surface to retrofit per year and per CECB category
        S_tR_G = S_df * RR_tot * RR_G
        S_tR_F = S_df * RR_tot * RR_F
        S_tR_E = S_df * RR_tot * RR_E
        S_tR_D = S_df * RR_tot * RR_D
        S_tR_C = S_df * RR_tot * RR_C
        S_tR_B = S_df * RR_tot * RR_B
        S_tR_A = S_df * RR_tot * RR_A
        
        #Simulation modules
        
            # Module to tune the proportion of A retrofit
        
        if x > 2019 and x % 2 == 0:
            if prop_G + diff_prop <= 1:
                prop_G += diff_prop
            if prop_F + diff_prop <= 1:
                prop_F += diff_prop
            if prop_E + diff_prop <= 1:
                prop_E += diff_prop
            if prop_D + diff_prop <= 1:
                prop_D += diff_prop
            if prop_C + diff_prop <= 1:
                prop_C += diff_prop
                
            # Module to tune the IDC_n
        if x == 2025:
            IDC_n = IDC_n + DIDC
        if x == 2035:
            IDC_n = IDC_n + DIDC
        if x == 2045:
            IDC_n = IDC_n + DIDC
            
            # Module to tune the RR_tot
        if x >= 2019 and RR_tot < RR_max:
            RR_tot += dif_RR
            
        RR_year.append(RR_tot)
        
        #--- Phase 1: Energy demand from the df ---#
        
        if len(SRE_G) == 0 and len(SRE_F) > 0:
            S_tR_F = S_tR_F + S_tR_G
        if len(SRE_F) == 0 and len(SRE_E) > 0:
            S_tR_E = S_tR_E + S_tR_F + S_tR_G
        if len(SRE_E) == 0 and len(SRE_D) > 0:
            S_tR_D = S_tR_D + S_tR_E + S_tR_F + S_tR_G
        if len(SRE_D) == 0 and len(SRE_C) > 0:
            S_tR_C = S_tR_C + S_tR_D + S_tR_E + S_tR_F + S_tR_G
        if len(SRE_C) == 0 and len(SRE_B) > 0:
            S_tR_B = S_tR_B + S_tR_C + S_tR_D + S_tR_E + S_tR_F + S_tR_G
        if len(SRE_B) == 0:
            S_tR_A = S_tR_A + S_tR_B + S_tR_C + S_tR_D + S_tR_E + S_tR_F + S_tR_G
            
            #For G buildings
        Surface_t.clear() #We clear surface_t
        IDC_t.clear() # We clear IDC_t
        exit = 0 #Set the exit element
        count = 0
            #Pick the building we will retrofit
        if len(SRE_G) > 0: #If there is some buildings in the G category
            while exit != 1: # While we don't say it is enough.
                if len(SRE_G) == 0: #if G is empty, we report the surface to retrofit to the next one.
                    S_tR_G = S_tR_G - sum(Surface_t) # We substract the surface in the list to the one to retrofit.
                    S_tR_F = S_tR_F + S_tR_G # We report the rest of the surface to retrofit to the next generation.
                    break
                number = random.randint(0,len(SRE_G)) # Generation of a random number
                if number == len(SRE_G): # If this number is equal to the length of the list, we reduce of 1.
                    number = number - 1
                if sum(Surface_t) + SRE_G[number] <= S_tR_G + S_tR_G * range: # If the sum of the surface + the building selected is smaller than the surface to retrofit + the range.
                    Surface_t.append(SRE_G[number]) # We include the building in the surface lsit
                    del SRE_G[number] # We can delete the building from the surface G
                    del IDC_G[number] # We can delete the building from the IDC G
                    count = 0 # We reset the count to 0
                else: # If the total surface is bigger
                    count += 1 # We increase he count by one
                if sum(Surface_t) >= S_tR_G - S_tR_G * range and sum(Surface_t) <= S_tR_G + S_tR_G * range: # If in the target surface
                    exit = 1 # We can go out
                if count == 50: # If we have failed 50 times
                    range += 0.01 # We increase the range by 1%
                    count = 0 # We reset the count to 0.
            
            #Energy demand of the new retrofit surface
            
        number_A = int(round(len(Surface_t) * prop_G))
        count = 0
        while count < number_A:
            SRE_A.append(Surface_t[0])
            number = random.randint(10, CECB_A)
            IDC_A.append(number)
            del Surface_t[0]
            count += 1
            
        while len(Surface_t) > 0:
            SRE_B.append(Surface_t[0])
            number = random.randint(CECB_A + 1, CECB_B)
            IDC_B.append(number)
            del Surface_t[0]
            
        #For F buildings
        Surface_t.clear()
        exit = 0
        count = 0
            #Pick the building we will retrofit
        if len(SRE_F) > 0:
            while exit != 1:
                if len(SRE_F) == 0:
                    S_tR_F = S_tR_F - sum(Surface_t)
                    S_tR_E = S_tR_E + S_tR_F
                    break
                number = random.randint(0,len(SRE_F))
                if number == len(SRE_F):
                    number = number - 1
                if sum(Surface_t) + SRE_F[number] <= S_tR_F + S_tR_F * range:
                    Surface_t.append(SRE_F[number])
                    del SRE_F[number]
                    del IDC_F[number]
                    count = 0
                else:
                    count += 1
                if sum(Surface_t) >= S_tR_F - S_tR_F * range and sum(Surface_t) <= S_tR_F + S_tR_F * range:
                    exit = 1
                if count == 50:
                    range += 0.01
                    count = 0
            
            #Energy demand of the new retrofit surface
            
        number_A = int(round(len(Surface_t) * prop_F))
        count = 0
        while count < number_A:
            SRE_A.append(Surface_t[0])
            number = random.randint(10, CECB_A)
            IDC_A.append(number)
            del Surface_t[0]
            count += 1
        count = 0
        while count < len(Surface_t):
            SRE_B.append(Surface_t[0])
            number = random.randint(CECB_A + 1, CECB_B)
            IDC_B.append(number)
            del Surface_t[0]
            count += 1
            
        #For E buildings
        Surface_t.clear()
        exit = 0
        count = 0
            #Pick the building we will retrofit
        if len(SRE_E) > 0:
            while exit != 1:
                if len(SRE_E) == 0:
                    S_tR_E = S_tR_E - sum(Surface_t)
                    S_tR_D = S_tR_D + S_tR_E
                    break
                number = random.randint(0,len(SRE_E))
                if number == len(SRE_E):
                    number = number - 1
                if sum(Surface_t) + SRE_E[number] <= S_tR_E + S_tR_E * range:
                    Surface_t.append(SRE_E[number])
                    del SRE_E[number]
                    del IDC_E[number]
                    count = 0
                else:
                    count += 1
                if sum(Surface_t) >= S_tR_E - S_tR_E * range and sum(Surface_t) <= S_tR_E + S_tR_E * range:
                    exit = 1
                if count == 50:
                    range += 0.01
                    count = 0
            
            #Energy demand of the new retrofit surface
            
        number_A = int(round(len(Surface_t) * prop_E))
        count = 0
        while count < number_A:
            SRE_A.append(Surface_t[0])
            number = random.randint(10, CECB_A)
            IDC_A.append(number)
            del Surface_t[0]
            count += 1
        count = 0
        while count < len(Surface_t):
            SRE_B.append(Surface_t[0])
            number = random.randint(CECB_A + 1, CECB_B)
            IDC_B.append(number)
            del Surface_t[0]
            count += 1
            
        #For D buildings
        Surface_t.clear()
        exit = 0
        count = 0
            #Pick the building we will retrofit
        if len(SRE_D) > 0:
            while exit != 1:
                if len(SRE_D) == 0:
                    S_tR_D = S_tR_D - sum(Surface_t)
                    S_tR_C = S_tR_C + S_tR_D
                    break
                number = random.randint(0,len(SRE_D))
                if number == len(SRE_D):
                    number = number - 1
                if sum(Surface_t) + SRE_D[number] <= S_tR_D + S_tR_D * range:
                    Surface_t.append(SRE_D[number])
                    del SRE_D[number]
                    del IDC_D[number]
                    count = 0
                else:
                    count += 1
                if sum(Surface_t) >= S_tR_D - S_tR_D * range and sum(Surface_t) <= S_tR_D + S_tR_D * range:
                    exit = 1
                if count == 50:
                    range += 0.01
                    count = 0
            
            #Energy demand of the new retrofit surface
            
        number_A = int(round(len(Surface_t) * prop_D))
        count = 0
        while count < number_A:
            SRE_A.append(Surface_t[0])
            number = random.randint(10, CECB_A)
            IDC_A.append(number)
            del Surface_t[0]
            count += 1
        count = 0
        while count < len(Surface_t):
            SRE_B.append(Surface_t[0])
            number = random.randint(CECB_A + 1, CECB_B)
            IDC_B.append(number)
            del Surface_t[0]
            count += 1
        
        #For C buildings
        Surface_t.clear()
        exit = 0
        count = 0
            #Pick the building we will retrofit
        if len(SRE_C) > 0:
            while exit != 1:
                if len(SRE_C) == 0:
                    S_tR_C = S_tR_C - sum(Surface_t)
                    S_tR_B = S_tR_B + S_tR_C
                    break
                number = random.randint(0,len(SRE_C))
                if number == len(SRE_C):
                    number = number - 1
                if sum(Surface_t) + SRE_C[number] <= S_tR_C + S_tR_C * range:
                    Surface_t.append(SRE_C[number])
                    del SRE_C[number]
                    del IDC_C[number]
                    count = 0
                else:
                    count += 1
                if sum(Surface_t) >= S_tR_C - S_tR_C * range and sum(Surface_t) <= S_tR_C + S_tR_C * range:
                    exit = 1
                if count == 50:
                    range += 0.01
                    count = 0
            
            #Energy demand of the new retrofit surface
            
        number_A = int(round(len(Surface_t) * prop_C))
        count = 0
        while count < number_A:
            SRE_A.append(Surface_t[0])
            number = random.randint(10, CECB_A)
            IDC_A.append(number)
            del Surface_t[0]
            count += 1
        count = 0
        while count < len(Surface_t):
            SRE_B.append(Surface_t[0])
            number = random.randint(CECB_A + 1, CECB_B)
            IDC_B.append(number)
            del Surface_t[0]
            count += 1
            
        #For B buildings
        Surface_t.clear()
        exit = 0
        count = 0
            #Pick the building we will retrofit
        if len(SRE_B) > 0:
            while exit != 1:
                if len(SRE_B) == 0:
                    S_tR_B = S_tR_B - sum(Surface_t)
                    S_tR_A = S_tR_A + S_tR_B
                    break
                number = random.randint(0,len(SRE_B))
                if number == len(SRE_B):
                    number = number - 1
                if sum(Surface_t) + SRE_B[number] <= S_tR_B + S_tR_B * range:
                    Surface_t.append(SRE_B[number])
                    del SRE_B[number]
                    del IDC_B[number]
                    count = 0
                else:
                    count += 1
                if sum(Surface_t) >= S_tR_B - S_tR_B * range and sum(Surface_t) <= S_tR_B + S_tR_B * range:
                    exit = 1
                if count == 50:
                    range += 0.01
                    count = 0
            
            #Energy demand of the new retrofit surface
            
        number_A = int(round(len(Surface_t) * prop_B))
        count = 0
        while count < number_A:
            SRE_A.append(Surface_t[0])
            number = random.randint(10, CECB_A)
            IDC_A.append(number)
            del Surface_t[0]
            count += 1
        count = 0
            
        #Energy demand for X year
        count = 0
        for step in SRE_G:
            E_df = E_df + step * IDC_G[count]
            count += 1
        count = 0
        for step in SRE_F:
            E_df = E_df + step * IDC_F[count]
            count += 1
        count = 0
        for step in SRE_E:
            E_df = E_df + step * IDC_E[count]
            count += 1
        count = 0
        for step in SRE_D:
            E_df = E_df + step * IDC_D[count]
            count += 1
        count = 0
        for step in SRE_C:
            E_df = E_df + step * IDC_C[count]
            count += 1
        count = 0
        for step in SRE_B:
            E_df = E_df + step * IDC_B[count]
            count += 1
        count = 0
        for step in SRE_A:
            E_df = E_df + step * IDC_A[count]
            count += 1
        
        E_df_year.append(E_df)
        
        #Energy profile of the df
            #Sum of the surface in every CECB class
        sum_A = sum(SRE_A)
        sum_B = sum(SRE_B)
        sum_C = sum(SRE_C)
        sum_D = sum(SRE_D)
        sum_E = sum(SRE_E)
        sum_F = sum(SRE_F)
        sum_G = sum(SRE_G)
            #Sum of the surface over the overall surface in order to have the percentage
        per_CECB = [sum_A/S_df, sum_B/S_df, sum_C/S_df, sum_D/S_df, sum_E/S_df, sum_F/S_df, sum_G/S_df]
            #We store the column in a df in order to display these percentage later
        df_per_cat[x] = per_CECB
            #Measurement of the average
        count = 0
        E_t = 0
        for step in SRE_G:
            E_t = E_t + step* IDC_G[count]
            count += 1
        if len(SRE_G) > 0 :
            CECB_G_t = E_t / sum(SRE_G)
        count = 0
        E_t = 0
        for step in SRE_F:
            E_t = E_t + step* IDC_F[count]
            count += 1
        if len(SRE_F) > 0 :
            CECB_F_t = E_t / sum(SRE_F)
        count = 0
        E_t = 0
        for step in SRE_E:
            E_t = E_t + step* IDC_E[count]
            count += 1
        if len(SRE_E) > 0 :
            CECB_E_t = E_t / sum(SRE_E)
        count = 0
        E_t = 0
        for step in SRE_D:
            E_t = E_t + step* IDC_D[count]
            count += 1
        if len(SRE_D) > 0 :
            CECB_D_t = E_t / sum(SRE_D)
        count = 0
        E_t = 0
        for step in SRE_C:
            E_t = E_t + step* IDC_C[count]
            count += 1
        if len(SRE_E) > 0 :
            CECB_C_t = E_t / sum(SRE_C)
        count = 0
        E_t = 0
        for step in SRE_B:
            E_t = E_t + step* IDC_B[count]
            count += 1
        if len(SRE_B) > 0 :
            CECB_B_t = E_t / sum(SRE_B)
        count = 0
        E_t = 0
        for step in SRE_A:
            E_t = E_t + step* IDC_A[count]
            count += 1
        if len(SRE_A) > 0 :
            CECB_A_t = E_t / sum(SRE_A)
        
        
                        #--- 2nd Phase: Energy demand from the surface which is not in the df---#
            
        S_A = S_add * per_CECB[0]
        S_B = S_add * per_CECB[1]
        S_C = S_add * per_CECB[2]
        S_D = S_add * per_CECB[3]
        S_E = S_add * per_CECB[4]
        S_F = S_add * per_CECB[5]
        S_G = S_add * per_CECB[6]
        
        E_add = S_A * CECB_A_t + S_B * CECB_B_t + S_C * CECB_C_t + S_D * CECB_D_t + S_E * CECB_E_t + S_F * CECB_F_t + S_G * CECB_G_t
        E_add_year.append(E_add)
        
        
                        #--- 3rd Phase: Energy demand from the new surface built from 2011
        
        S_n_t = S_tot - S_df - S_add - S_n
        S_n = S_n + S_n_t
        E_n = E_n + S_n_t * IDC_n
        E_n_year.append(E_n)
        
        
        
        x += 1 # Happy New Year !
        
        
    #Agregation of the energy
    count = 0
    for element in E_df_year:
        E_S.append((element + E_add_year[count] + E_n_year[count])/S_tot_year[count])
        E_tot.append(element + E_add_year[count] + E_n_year[count])
        count += 1
    
    df_kwh_m2[sim] = E_tot
    df_RR[sim] = RR_year

    
    #DIDC -= 1
    #RR_max += 0.001  
    #RR_tot_ini += 0.001
    #diff_prop += 0.005
    #Pop_rate += 0.001
    
    #print(df_per_cat)
    
    sim += 1
#print(IDC_n)

#print(df_RR)
#print(df_kwh_m2)  

        

31
    Year             0             1             2             3  \
0   2011  2.705684e+09  2.706729e+09  2.706302e+09  2.706363e+09   
1   2012  2.692129e+09  2.692747e+09  2.690835e+09  2.692494e+09   
2   2013  2.678878e+09  2.677235e+09  2.676390e+09  2.678717e+09   
3   2014  2.666312e+09  2.662838e+09  2.662038e+09  2.663882e+09   
4   2015  2.653899e+09  2.648981e+09  2.647783e+09  2.650346e+09   
5   2016  2.641097e+09  2.634948e+09  2.634466e+09  2.636678e+09   
6   2017  2.629487e+09  2.620908e+09  2.620810e+09  2.622987e+09   
7   2018  2.616704e+09  2.607579e+09  2.607215e+09  2.609466e+09   
8   2019  2.603620e+09  2.594947e+09  2.594105e+09  2.595568e+09   
9   2020  2.587903e+09  2.579344e+09  2.578751e+09  2.580053e+09   
10  2021  2.569897e+09  2.561562e+09  2.559047e+09  2.562282e+09   
11  2022  2.550533e+09  2.542701e+09  2.539615e+09  2.543676e+09   
12  2023  2.530193e+09  2.521913e+09  2.518674e+09  2.522608e+09   
13  2024  2.506817e+09  2.498026e+09  2.49473

In [3]:
#Excel export
export_excel = df_kwh_m2.to_excel (r'kwhm2.xlsx', index = None, header=True)
#export_excel = df_per_cat.to_excel (r'kwhm2.xlsx', index = None, header=True)